In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_file_path = '../input/digit-recognizer/train.csv'

train_data = pd.read_csv(train_file_path) 


In [ ]:
test_file_path = '../input/digit-recognizer/test.csv'

test_data = pd.read_csv(test_file_path) 
train_data.head()
test_data.head()


In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
#input_features = list(preprocessed_train_df.columns)
import matplotlib.pyplot as plt
import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
train_data.isnull().values.any()
train_data.shape
df =train_data['label']
df.head(10)
df.dtype
train_data.drop('label', inplace=True, axis=1)
train_data.head(14)


train_data = train_data.values
df = df.values
test_data = test_data.values

train_data = train_data.reshape(-1, 28, 28, 1) / 255.0
test_data = test_data.reshape(-1, 28, 28, 1) / 255.0

In [ ]:
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

train_X, val_X, train_y, val_y = train_test_split(train_data, df, random_state = 0)
model = keras.Sequential([
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same',
                  # give the input dimensions in the first layer
                  # [height, width, color channels(RGB)]
                  input_shape=[28,28,1]),
     # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Third Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(512, activation='relu', ),
    layers.Dropout(0.2),
   
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(10,'softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
)

history = model.fit(
   train_X, train_y,
    validation_data=(val_X, val_y),
    batch_size=64,
    epochs=5,
    callbacks=[early_stopping], # put your callbacks in a list
    #verbose=0, 
)


In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))

In [ ]:
label = model.predict(test_data)

In [ ]:

label=np.rint(label)
arr = np.arange(1, 28001)

label = np.argmax(label, axis=1)

In [ ]:
#test_data.head(10)
#print( len(test_data.index))
a = np.arange(1, 28001)
print(a)
output = pd.DataFrame({'ImageId': arr,
                       'Label': label})
output.head()

In [ ]:
output.to_csv('submission.csv', index=False)